# Deep Learning Final Project - Study

In [1]:
%pwd

'/home/by783/Self_Jupyter/DL_Final_Project'

In [2]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time

In [3]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [4]:
torch.cuda.is_available()

True

In [5]:
loader_image_path='/scratch/by783/DL_Final/ssl_data_96'
loader_batch_size=64

In [6]:
def image_loader(path, batch_size):
    transform = transforms.Compose(
        [
            transforms.ToTensor()
        ]
    )
    sup_train_data = datasets.ImageFolder('{}/{}/train'.format(path, 'supervised'), transform=transform)
    sup_val_data = datasets.ImageFolder('{}/{}/val'.format(path, 'supervised'), transform=transform)
    unsup_data = datasets.ImageFolder('{}/{}/'.format(path, 'unsupervised'), transform=transform)
    # source code: https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
    # Main idea:
    data_loader_sup_train = torch.utils.data.DataLoader(
        sup_train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_sup_val = torch.utils.data.DataLoader(
        sup_val_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_unsup = torch.utils.data.DataLoader(
        unsup_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    
    print('sup_train_data.class_to_idx==sup_val_data.class_to_idx: ',sup_train_data.class_to_idx==sup_val_data.class_to_idx)
    
    return data_loader_sup_train, data_loader_sup_val, data_loader_unsup, sup_train_data.class_to_idx

# Data Loader

https://stackoverflow.com/questions/51906144/pytorch-image-label

source code: https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html



datasets.ImageFolder是一个Class <br>
它会把data load进来<br>

    格式: root下面的folder name就是class name
    
    root/class_x/xxx.ext
    root/class_x/xxy.ext
    root/class_x/xxz.ext
    root/class_y/123.ext
    root/class_y/nsdf3.ext
    root/class_y/asd932_.ext

期中 _find_classes会把class label转化为数字index<br>

    classname会被sort
     if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}

参考Draft:

    datasets.ImageFolder[batch_i] 可以把第i个batched sample提取出来
    
        如果sample为[channel,wideth,height]
    
        这个东西返回的是一个len=2的list: [[batchsize,channel,wideth,height]-- tensor:samples,[batchsize] --tensor: batched idxes ]
    
    
     dataloader是一个iterator
     
         for item in dataloader:
             item拿出来的就是 datasets.ImageFolder[batch_i] 这个东西

In [7]:
data_loader_sup_train, data_loader_sup_val, data_loader_unsup, class_to_idx_dict = image_loader(loader_image_path,loader_batch_size)

sup_train_data.class_to_idx==sup_val_data.class_to_idx:  True


In [8]:
#class_to_idx_dict

In [9]:
# build a idx to class dict
idx_to_class_dict={}
for k,v in class_to_idx_dict.items():
    idx_to_class_dict[v]=k

In [11]:
for sample_batched in data_loader_sup_train:
    print( len(sample_batched) )
    break
'''
for i_batch, sample_batched in enumerate(data_loader_sup_train):
    print(i_batch, len(sample_batched) )
    if i_batch>=0: break
'''
# use the above code to get batch number

2


'\nfor i_batch, sample_batched in enumerate(data_loader_sup_train):\n    print(i_batch, len(sample_batched) )\n    if i_batch>=0: break\n'

In [12]:
sample_batched[0].size()

torch.Size([64, 3, 96, 96])

In [13]:
sample_batched[1].size()

torch.Size([64])

In [ ]:
#idx_to_class_dict

# Training

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [24]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  0.4.1
Torchvision Version:  0.2.2


In [14]:
dataloaders={}
dataloaders['train'],dataloaders['val']=data_loader_sup_train, data_loader_sup_val

In [15]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [22]:
model_name="vgg"
num_classes = 1000
num_epochs = 15

In [18]:
feature_extract = True
# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params

In [19]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [21]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [25]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to /home/by783/.torch/models/vgg11_bn-6002323d.pth
100%|██████████| 531503671/531503671 [00:07<00:00, 70461675.28it/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=

In [27]:
input_size

224

In [26]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [28]:
def image_loader(path, batch_size):
    transform = transforms.Compose(
        [
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )
    sup_train_data = datasets.ImageFolder('{}/{}/train'.format(path, 'supervised'), transform=transform)
    sup_val_data = datasets.ImageFolder('{}/{}/val'.format(path, 'supervised'), transform=transform)
    unsup_data = datasets.ImageFolder('{}/{}/'.format(path, 'unsupervised'), transform=transform)
    # source code: https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
    # Main idea:
    data_loader_sup_train = torch.utils.data.DataLoader(
        sup_train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_sup_val = torch.utils.data.DataLoader(
        sup_val_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_unsup = torch.utils.data.DataLoader(
        unsup_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    
    print('sup_train_data.class_to_idx==sup_val_data.class_to_idx: ',sup_train_data.class_to_idx==sup_val_data.class_to_idx)
    
    return data_loader_sup_train, data_loader_sup_val, data_loader_unsup, sup_train_data.class_to_idx

In [29]:
data_loader_sup_train, data_loader_sup_val, data_loader_unsup, class_to_idx_dict = image_loader(loader_image_path,loader_batch_size)

sup_train_data.class_to_idx==sup_val_data.class_to_idx:  True


In [30]:
dataloaders={}
dataloaders['train'],dataloaders['val']=data_loader_sup_train, data_loader_sup_val

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
device

device(type='cuda', index=0)

In [33]:
model_ft = model_ft.to(device)

In [34]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 6.2418 Acc: 0.0735
val Loss: 5.4593 Acc: 0.2354

Epoch 1/14
----------
train Loss: 4.9675 Acc: 0.2333
val Loss: 4.4769 Acc: 0.3123

Epoch 2/14
----------
train Loss: 4.1627 Acc: 0.3149
val Loss: 3.8739 Acc: 0.3505

Epoch 3/14
----------
train Loss: 3.6635 Acc: 0.3588
val Loss: 3.5120 Acc: 0.3721

Epoch 4/14
----------


# Draft

## Data Loader

In [21]:

data_loader_try, _, _ = image_loader(loader_image_path,1)

In [29]:
for i_batch, sample_batched in enumerate(data_loader_try):
    print(i_batch, len(sample_batched) )
    if i_batch>=0: break

0 2


In [93]:
sample_batched

[tensor([[[[0.4392, 0.4392, 0.4314,  ..., 0.3294, 0.3569, 0.3412],
           [0.4392, 0.4431, 0.4471,  ..., 0.2941, 0.3451, 0.3412],
           [0.4314, 0.4392, 0.4431,  ..., 0.2510, 0.3294, 0.3647],
           ...,
           [0.2824, 0.2667, 0.2667,  ..., 0.3176, 0.3176, 0.3137],
           [0.2706, 0.2627, 0.2471,  ..., 0.3176, 0.3176, 0.3020],
           [0.2510, 0.2431, 0.2275,  ..., 0.3059, 0.3059, 0.2824]],
 
          [[0.4824, 0.4824, 0.4706,  ..., 0.3490, 0.3843, 0.3725],
           [0.4824, 0.4863, 0.4863,  ..., 0.3176, 0.3686, 0.3686],
           [0.4745, 0.4824, 0.4824,  ..., 0.2745, 0.3608, 0.3922],
           ...,
           [0.3686, 0.3529, 0.3412,  ..., 0.4392, 0.4392, 0.4353],
           [0.3490, 0.3373, 0.3216,  ..., 0.4392, 0.4392, 0.4275],
           [0.3294, 0.3176, 0.2980,  ..., 0.4314, 0.4314, 0.4078]],
 
          [[0.4980, 0.4980, 0.4784,  ..., 0.0902, 0.0824, 0.0471],
           [0.4980, 0.4941, 0.4941,  ..., 0.0667, 0.0784, 0.0549],
           [0.4824, 0.49

In [76]:
sample_batched[1]

tensor([902])

In [37]:
transform = transforms.Compose(
    [
        transforms.ToTensor()
    ])

In [82]:
sup_train_data = datasets.ImageFolder('{}/{}/train'.format(loader_image_path, 'supervised'), transform=transform)

In [117]:
sup_train_data[0]

(tensor([[[0.7608, 0.7725, 0.7804,  ..., 0.5373, 0.5569, 0.5412],
          [0.7647, 0.7765, 0.7843,  ..., 0.5451, 0.5373, 0.5451],
          [0.7725, 0.7804, 0.7922,  ..., 0.5569, 0.5294, 0.5451],
          ...,
          [0.8431, 0.7961, 0.8196,  ..., 0.0667, 0.0706, 0.0627],
          [0.8196, 0.8392, 0.8314,  ..., 0.0549, 0.0706, 0.0863],
          [0.8471, 0.8235, 0.8000,  ..., 0.0588, 0.0706, 0.0902]],
 
         [[0.8000, 0.8118, 0.8196,  ..., 0.5098, 0.5294, 0.5216],
          [0.8039, 0.8157, 0.8235,  ..., 0.5216, 0.5137, 0.5255],
          [0.8118, 0.8196, 0.8314,  ..., 0.5333, 0.5059, 0.5176],
          ...,
          [0.8118, 0.7647, 0.7882,  ..., 0.0353, 0.0392, 0.0314],
          [0.7882, 0.8078, 0.8000,  ..., 0.0235, 0.0392, 0.0549],
          [0.8157, 0.7922, 0.7686,  ..., 0.0275, 0.0392, 0.0588]],
 
         [[0.8078, 0.8157, 0.8235,  ..., 0.4471, 0.4588, 0.4431],
          [0.8118, 0.8235, 0.8314,  ..., 0.4667, 0.4510, 0.4510],
          [0.8196, 0.8275, 0.8392,  ...,

In [83]:
sup_val_data = datasets.ImageFolder('{}/{}/val'.format(loader_image_path, 'supervised'), transform=transform)

In [85]:
# https://discuss.pytorch.org/t/questions-about-imagefolder/774
sup_train_data.class_to_idx==sup_val_data.class_to_idx

True

In [ ]:
sup_val_data = datasets.ImageFolder('{}/{}/val'.format(path, 'supervised'), transform=transform)

In [55]:

data_loader_sup_train = torch.utils.data.DataLoader(
        sup_train_data,
        batch_size=1,
        shuffle=True,
        num_workers=0)